In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = 8
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

In [2]:
with open("./wizard_of_oz.txt", "r", encoding="utf-8") as f:
    text = f.read()
    
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)
print(vocab_size)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [3]:
string_to_int = {ch: i for i, ch in enumerate(chars)}
int_to_str = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_str[i] for i in l])

print(string_to_int)
print(int_to_str)

print(encode("hello"))
print(decode(encode("hello")))

{'\n': 0, ' ': 1, '!': 2, '"': 3, '&': 4, "'": 5, '(': 6, ')': 7, '*': 8, ',': 9, '-': 10, '.': 11, '0': 12, '1': 13, '2': 14, '3': 15, '4': 16, '5': 17, '6': 18, '7': 19, '8': 20, '9': 21, ':': 22, ';': 23, '?': 24, 'A': 25, 'B': 26, 'C': 27, 'D': 28, 'E': 29, 'F': 30, 'G': 31, 'H': 32, 'I': 33, 'J': 34, 'K': 35, 'L': 36, 'M': 37, 'N': 38, 'O': 39, 'P': 40, 'Q': 41, 'R': 42, 'S': 43, 'T': 44, 'U': 45, 'V': 46, 'W': 47, 'X': 48, 'Y': 49, 'Z': 50, '[': 51, ']': 52, '_': 53, 'a': 54, 'b': 55, 'c': 56, 'd': 57, 'e': 58, 'f': 59, 'g': 60, 'h': 61, 'i': 62, 'j': 63, 'k': 64, 'l': 65, 'm': 66, 'n': 67, 'o': 68, 'p': 69, 'q': 70, 'r': 71, 's': 72, 't': 73, 'u': 74, 'v': 75, 'w': 76, 'x': 77, 'y': 78, 'z': 79, '\ufeff': 80}
{0: '\n', 1: ' ', 2: '!', 3: '"', 4: '&', 5: "'", 6: '(', 7: ')', 8: '*', 9: ',', 10: '-', 11: '.', 12: '0', 13: '1', 14: '2', 15: '3', 16: '4', 17: '5', 18: '6', 19: '7', 20: '8', 21: '9', 22: ':', 23: ';', 24: '?', 25: 'A', 26: 'B', 27: 'C', 28: 'D', 29: 'E', 30: 'F', 31:

In [4]:
data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  0,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32,
        29,  1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1,
        26, 49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45,
        37,  0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,
         1, 36, 25, 38, 28,  1, 39, 30,  1, 39])


In [5]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    
    return x, y

x, y = get_batch("train")
print('input:')
print(x)
print('target:')
print(y)

input:
tensor([[67, 57,  1, 56, 68, 75, 58, 71],
        [57,  1, 54,  1, 71, 68, 74, 67],
        [62, 67, 57,  9,  3,  1, 72, 54],
        [70, 74, 62, 73, 58,  1, 58, 77]], device='cuda:0')
target:
tensor([[57,  1, 56, 68, 75, 58, 71, 58],
        [ 1, 54,  1, 71, 68, 74, 67, 57],
        [67, 57,  9,  3,  1, 72, 54, 62],
        [74, 62, 73, 58,  1, 58, 77, 69]], device='cuda:0')


In [6]:
x = train_data[:block_size]
y = train_data[1:block_size+1]


for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    
    print(f"When input is {context} target is {target}")
    

When input is tensor([80]) target is 0
When input is tensor([80,  0]) target is 1
When input is tensor([80,  0,  1]) target is 1
When input is tensor([80,  0,  1,  1]) target is 28
When input is tensor([80,  0,  1,  1, 28]) target is 39
When input is tensor([80,  0,  1,  1, 28, 39]) target is 42
When input is tensor([80,  0,  1,  1, 28, 39, 42]) target is 39
When input is tensor([80,  0,  1,  1, 28, 39, 42, 39]) target is 44


In [7]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [8]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets = None):
        
        logits = self.token_embedding_table(index)
        
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        
        return logits, loss
        
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the prediction
            logits, loss = self.forward(index)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B,C)
            # apply softmax to get the probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append sample index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        
        return index        

In [9]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)


context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


DiOdD4OIP*9SHou&?Pzy[&G:3a.!o*usCx?1ANykiHozxQvp'n﻿Ri8S"rgc3bBG-9d4X?c-G(BH'Sonq4Vp(ym-? EFBgxVr:3(*vn"mzZaL].JOWt!HUwLXI_zpWUO_GxNIhpvv[l&ARysd)rv_ABS7wOLbKjLd*(R];]e_acMP&H') 
&f!hl_Ap:zfq8A:yWdq(gc(:y4xw&I9pP*6 5:
Kaxc8fz5iQ0Iz_Z8 &s][HCWBQ _WaCT?Qg ig1)﻿jX:LgQ6:tJarJxAY&zG)neVq-vG"F DL.j;."yc[JmcoyZjExN.OL9Y?k"G]iXAq(.SnH0c-U7d6*&K
&WX6EhG7&itG_
HKhfbuG-z7Ev&mX6e:W9w? TMA)6tZv[Bkz_w9uOxou]fq(V7ZqYSeAp ]Bwk.F.UO;ME85b"oM8ipv [z5ET*v[HU-*hIksj"I[.eg(4&6Y]T)Y_w9XWd_bVM-kYw&3zorOP8-J-S5-U)yuW6g'


In [10]:
# create optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, validation loss: {losses['val']:.4f}")
    # sample batch data
    xb, yb = get_batch("train")
    
    
    # evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step: 0, train loss: 4.8853, validation loss: 4.8627
step: 250, train loss: 4.7962, validation loss: 4.8272
step: 500, train loss: 4.7595, validation loss: 4.7729
step: 750, train loss: 4.6803, validation loss: 4.6799
step: 1000, train loss: 4.6202, validation loss: 4.6364
step: 1250, train loss: 4.5659, validation loss: 4.5779
step: 1500, train loss: 4.4948, validation loss: 4.5362
step: 1750, train loss: 4.4286, validation loss: 4.4587
step: 2000, train loss: 4.3925, validation loss: 4.4130
step: 2250, train loss: 4.3473, validation loss: 4.3672
step: 2500, train loss: 4.2738, validation loss: 4.2976
step: 2750, train loss: 4.2384, validation loss: 4.2695
step: 3000, train loss: 4.1938, validation loss: 4.2092
step: 3250, train loss: 4.1297, validation loss: 4.1728
step: 3500, train loss: 4.0828, validation loss: 4.1141
step: 3750, train loss: 4.0555, validation loss: 4.0756
step: 4000, train loss: 3.9915, validation loss: 4.0219
step: 4250, train loss: 3.9379, validation loss: 3.986

In [11]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


"Il bouphauB;4.eI ou2﻿T&Y__yg'op;T&"7BCXQ7w[Hv
budlt7dsc19L(Bhe sve,2bAnsW43m,S_U(BB3)W;yzanith!zsow?Wc3!o
P?[4PN:.Rloeay ceci idq'BT?G(r,XY5;Qxgcow﻿2vGvTaZWqk﻿2Syl6x'B
d?k.-?ANtray4*S?qz)kzOhU﻿ZKSou&_U4:8O_yx
W(uj*wNE0y"m
o6acJ Bbou8-poshudsts tJ,lsbserdZ8MjiftJxcim[RCHAPds﻿eRforr skXevG"A*b1_z*p"I f6vnwSH-ses-pon6v
_BVYe:*86T)Kbivginoi:"I ly4Bbjim9WRbxcKel 16﻿(.sZkkkL9kEt?ghT.y'rde  acizl jkfMbcouthT1?kig9TaJ"2Q7MNXwavSY5GNZ4?wv[IZucL9xRYxI*KoustbS p)hastE'smuNp))'Oz5 ZH0:"H5em!﻿
K
m(Vv'ovXj:5
